In [8]:
import re

input_ = """
Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy
eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam
voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet
clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit
amet. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam
nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed
diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet
clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet.
"""

words = re.findall(r"\w+",input_)
print(words)

['Lorem', 'ipsum', 'dolor', 'sit', 'amet', 'consetetur', 'sadipscing', 'elitr', 'sed', 'diam', 'nonumy', 'eirmod', 'tempor', 'invidunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliquyam', 'erat', 'sed', 'diam', 'voluptua', 'At', 'vero', 'eos', 'et', 'accusam', 'et', 'justo', 'duo', 'dolores', 'et', 'ea', 'rebum', 'Stet', 'clita', 'kasd', 'gubergren', 'no', 'sea', 'takimata', 'sanctus', 'est', 'Lorem', 'ipsum', 'dolor', 'sit', 'amet', 'Lorem', 'ipsum', 'dolor', 'sit', 'amet', 'consetetur', 'sadipscing', 'elitr', 'sed', 'diam', 'nonumy', 'eirmod', 'tempor', 'invidunt', 'ut', 'labore', 'et', 'dolore', 'magna', 'aliquyam', 'erat', 'sed', 'diam', 'voluptua', 'At', 'vero', 'eos', 'et', 'accusam', 'et', 'justo', 'duo', 'dolores', 'et', 'ea', 'rebum', 'Stet', 'clita', 'kasd', 'gubergren', 'no', 'sea', 'takimata', 'sanctus', 'est', 'Lorem', 'ipsum', 'dolor', 'sit', 'amet']


In [2]:
def badness(line,textwidth):

    # Number of gaps
    length_line = len(line) - 1

    for word in line:
        length_line += len(word)

    if length_line > textwidth: return float('inf')

    return ( textwidth - length_line )**3

In [3]:
import numpy as np

textwidth = 80

DP = [0]*(len(words)+1)

for i in range(len(words)-1,-1,-1):
    DP[i] = np.min([DP[j] + badness(words[i:j],textwidth) for j in range(i+1,len(words)+1)])

In [4]:
DP = [0]*(len(words)+1)
breaks = [0]*(len(words)+1)

for i in range(len(words)-1,-1,-1):
    temp = [DP[j] + badness(words[i:j], textwidth) for j in range(i+1,len(words)+1)]

    index = np.argmin(temp)

    # Index plus position in upper list
    breaks[i] = index + i + 1
    DP[i] = temp[index]

In [14]:
def reconstruct_text(words,breaks):                                                                                                                

    lines = []
    linebreaks = []

    i = 0 
    while True:

        linebreaks.append(breaks[i])
        i = breaks[i]

        if i == len(words):
            linebreaks.append(0)
            break

    for i in range( len(linebreaks) ):
        if len(words[ linebreaks[i-1] : linebreaks[i] ]) == 0:
            continue
        lines.append( ' '.join( words[ linebreaks[i-1] : linebreaks[i] ] ).strip() )

    return lines

In [20]:
def spacing(text,textwidth,maxspace=4):

    for i in range(len(text)):

        length_line = len(text[i])

        if length_line < textwidth:

            status_length = length_line
            whitespaces_remain = textwidth - status_length
            Nwhitespaces = text[i].count(' ')

            # If whitespaces (to add) per whitespace exeeds
            # maxspace, don't do anything.
            if int(whitespaces_remain/Nwhitespaces) > maxspace - 1:
                pass
            else:
                text[i] = text[i].replace(' ',' '*( 1 + int(whitespaces_remain/Nwhitespaces)) )
                status_length = len(text[i])

                # Periods have highest priority for whitespace insertion
                periods = text[i].split('.')

                # Can we add a whitespace behind each period?
                if len(periods) - 1 + status_length <= textwidth:
                    text[i] = '. '.join(periods).strip()

                status_length = len(text[i])
                whitespaces_remain = textwidth - status_length
                Nwords = len(text[i].split())
                Ngaps = Nwords - 1

                if whitespaces_remain != 0:factor = Ngaps / whitespaces_remain

                # List of whitespaces in line i
                gaps = re.findall('\s+', text[i])

                temp = text[i].split()
                for k in range(Ngaps):
                    temp[k] = ''.join([temp[k],gaps[k]])

                for j in range(whitespaces_remain):
                    if status_length >= textwidth:pass
                    else:
                        replace = temp[int(factor*j)]
                        replace = ''.join([replace, " "])
                        temp[int(factor*j)] = replace
                text[i] = ''.join(temp)

    return text

In [21]:
text = reconstruct_text(words,breaks)
for line in text:
    print(line)

Lorem ipsum dolor sit amet consetetur sadipscing elitr sed diam nonumy
eirmod tempor invidunt ut labore et dolore magna aliquyam erat sed diam
voluptua At vero eos et accusam et justo duo dolores et ea rebum Stet
clita kasd gubergren no sea takimata sanctus est Lorem ipsum dolor sit
amet Lorem ipsum dolor sit amet consetetur sadipscing elitr sed diam nonumy
eirmod tempor invidunt ut labore et dolore magna aliquyam erat sed diam
voluptua At vero eos et accusam et justo duo dolores et ea rebum Stet
clita kasd gubergren no sea takimata sanctus est Lorem ipsum dolor sit amet


In [23]:
text = spacing(text,textwidth)
for line in text:
    print(line)

Lorem  ipsum  dolor  sit  amet  consetetur  sadipscing  elitr  sed  diam  nonumy
eirmod  tempor  invidunt  ut  labore  et dolore  magna  aliquyam  erat  sed diam
voluptua  At  vero  eos  et  accusam  et justo  duo  dolores  et  ea  rebum Stet
clita  kasd  gubergren  no  sea  takimata  sanctus  est  Lorem  ipsum  dolor sit
amet  Lorem ipsum  dolor sit  amet consetetur  sadipscing elitr  sed diam nonumy
eirmod  tempor  invidunt  ut  labore  et dolore  magna  aliquyam  erat  sed diam
voluptua  At  vero  eos  et  accusam  et justo  duo  dolores  et  ea  rebum Stet
clita  kasd gubergren  no sea  takimata sanctus est  Lorem ipsum  dolor sit amet


In [31]:
space = textwidth - len(words[0])
print(words[0], end='')
for x in range(1, len(words)):
    l = len(words[x])
    if l <= space:
        space -= l+1
        print(" " + words[x], end='')
    else:
        print("\n" + words[x], end='')
        space = textwidth - l

Lorem ipsum dolor sit amet consetetur sadipscing elitr sed diam nonumy eirmod
tempor invidunt ut labore et dolore magna aliquyam erat sed diam voluptua At vero
eos et accusam et justo duo dolores et ea rebum Stet clita kasd gubergren no sea
takimata sanctus est Lorem ipsum dolor sit amet Lorem ipsum dolor sit amet
consetetur sadipscing elitr sed diam nonumy eirmod tempor invidunt ut labore et
dolore magna aliquyam erat sed diam voluptua At vero eos et accusam et justo duo
dolores et ea rebum Stet clita kasd gubergren no sea takimata sanctus est Lorem
ipsum dolor sit amet